# CSCI5523 Group7 Project on Gendered Wording in IT Job Posting

### Step 1. Data Import and Preprocessing for Topic Modeling and Linguistic Feature Extraction

In [ ]:
import os
os.chdir(r'C:\Users\kwonr\Dropbox\JobAd_Gender\Analysis')

import pymysql
import pandas as pd

##DB connection - connect()

posting = pymysql.connect( 
    user='root', 
    passwd='0825', 
    host='127.0.0.1', 
    db='job_posting', 
    charset='utf8',
    use_unicode=True
)

## cursor - cursor()
cursor = posting.cursor(pymysql.cursors.DictCursor)

## Table & Column Selection - SELECT
sql = "SELECT * FROM `_sb2j_clusr_cl_job`;"
cursor.execute(sql)
result = cursor.fetchall()

## Pandas Dataframe conversion
pd.set_option('display.max_columns', None)
df= pd.DataFrame(result)

# Understanding DataType
pd.set_option('display.max_rows', 100)
df.dtypes

# Make an index column for reference & later merge
df['index_col'] = df.index
df

import re
import os
import numpy as np
import nltk
import pickle
import tarfile
import chardet
from nltk.stem import PorterStemmer

p = '[a-zA-Z0-9]+'
stopwords={
    'max>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax':1,
    'edu':1,
    'subject':1,
    'com':1,
    'r<g':1,
    '_?w':1,
    'isc':1,
    'cx^':1,
    'usr':1,
    'uga':1,
    'sam':1,
    'mhz':1,
    'b8f':1,
    '34u':1,
    'pl+':1,
    '1993apr20':1,
    '1993apr15':1,
    'xterm':1,
    'utexas':1,
    'x11r5':1,
    'o+r':1,
    'iastate':1,
    'udel':1,
    'uchicago':1,
    '1993apr21':1,
    'uxa':1,
    'argic':1,
    'optilink':1,
    'imho':1,
    'umich':1,
    'openwindows':1,
    '1993apr19':1,
    '1993apr22':1,

}
vocabulary = {}
freq_threshold = 50
word_len_threshold = 2
ps=PorterStemmer()


def load_stopwords():
    # url: https://github.com/igorbrigadir/stopwords/blob/master/en/gensim.txt
    stopword_file = 'gensim_stopwords.txt'
    with open(stopword_file, mode='r', encoding='utf-8') as reader:
        for line in reader:
            word = line.strip()
            stopwords[word] = 1
    # print(stopwords[:50])


def load_vocab(path):
    vocab = []
    vocab_dict = {}
    with open(path, mode='r', encoding='utf-8') as reader:
        for line in reader:
            seg = line.strip().split()
            vocab.append(seg[0])
            vocab_dict[seg[0]] = len(vocab) # dict count from 1
    return vocab, vocab_dict


def clean_words(words):
    new_words = []
    for word in words:
        word = word.lower()
        if word in stopwords:
            continue
        word = word.strip('_\[\]\'\".,()*! #@~`\\%^&;:/-+=“”‘’<>{}|?$^').replace('isn\'t', '').replace('\'s', '').replace('\'re', '').replace('\'t', '').replace('\'ll', '').replace('\'m', '').replace('\'am', '').replace('\'ve', '').replace('\'d', '')
        segs = re.split('[()@.\-/#\\\\"`\[\]]', word)
        new_word = []
        for s in segs:
            seg=s
            # seg = ps.stem(seg)
            if seg not in stopwords and seg and len(seg) > word_len_threshold:
                new_word.append(seg)
        # word = ' '.join(new_word)
        # if word and len(word) > word_len_threshold:
        #     if word not in stopwords:
        #           new_words.append(word)
        new_words.extend(new_word)
    return new_words


def read_zip(path):
    data=[]
    if os.path.exists(path):
        for root, dirs, files in os.walk(path):
            for _file in files:
                # print(os.path.join(root, _file))
                content = []
                with open(os.path.join(root, _file), mode='rb') as reader:
                    content = reader.read()
                encode_info = chardet.detect(content)['encoding']
                if not encode_info:
                    encode_info = 'utf-8'
                content = content.decode(encoding=encode_info)
                content = content.split()
                data.append(' '.join(content))
    return data

def read_file(path):
    data = []
    with open(path, mode='r', encoding='utf-8') as reader:
        for line in reader:
            line = line.strip()
            data.append(line)
    return data

def parse_sent(line):
    delim = '[,.?!]'
    sents = []
    segs = re.split(delim, line)
    for seg in segs:
        if seg not in delim and len(seg)>5:
            sents.append(seg)
    return sents


def load_files_nvdm(path, vocab_dict,ifSplit=False):
    labels = {}
    data = []
    label_count=[]
    count = 0
    if os.path.exists(path):
        data = read_file(path)
    new_docs = []
    new_doc_sents = []
    for doc in data:
        new_doc = {}
        segs = doc.split()
        line = segs[-2::-1]
        label = segs[-1]
        label_count.append(label)
        words = line
        words = clean_words(words)
        for word in words:
            if word in vocab_dict:
                count += 1
                word_id = vocab_dict[word]
                if word_id in new_doc:
                    new_doc[word_id] += 1
                else:
                    new_doc[word_id] = 1

        # print(new_doc)
        if new_doc:
            new_docs.append(' '.join(['{}:{}'.format(item[0], item[1]) for item in new_doc.items()]))
            # print(new_docs[-1])

    idx = list(range(1, len(data)+1))
    # test_idx = list(np.random.choice(idx, 7531, replace=False))
    test_idx = []
    print('avg length:', count/len(data),count,len(data))
    doc_test=[]
    doc_train=[]
    for k, doc in enumerate(new_docs):
        line_doc = label_count[k]+' '+doc
        if k in test_idx:
            doc_test.append(line_doc)
        else:
            doc_train.append(line_doc)
    if doc_test:
        write_file('test.feat', doc_test)
    if doc_train:
        write_file('train.feat', doc_train)

    # with open('20news', mode='w', encoding='utf-8') as writer:
    #     writer.write('\n'.join(new_docs))
    # return data

def load_files_prodlda(path, vocab_dict):
    data = []
    if os.path.exists(path):
        data = read_file(path)
    new_docs = []
    for doc in data:
        segs = doc.split()
        line = segs[-2::-1]
        words = line
        words = clean_words(words)
        if words:
            new_docs.append(words)
    process_prolda(new_docs, vocab_dict, path+'.npy')

def write_file(path, data):
    with open(path, mode='w', encoding='utf-8') as writer:
        writer.write('\n'.join(data))


def produce_files(paths):
    data = []
    for path in paths:
        data = read_file(path)
        new_data = []
        for line in data:
            segs = line.split()
            doc = []
            for seg in segs[1:]:
                word_id, freq = seg.split(':')
                doc.extend([int(word_id)]*int(freq))
            doc = np.array(doc)
            new_data.append(doc)
        new_data = np.array(new_data)
        np.save(path, new_data)

def process_prolda(data, vocab, outputfile):
    new_docs = []
    for doc in data:
        new_doc = []
        for word in doc:
            if word in vocab:
                new_doc.append(vocab[word])
        new_doc = np.array(new_doc)
        new_docs.append(new_doc)
    new_docs = np.array(new_docs)
    print(new_docs.shape)
    np.save(outputfile, new_docs)

def clean_vocab(paths):
    label = []
    data = []
    for path in paths:
        data.extend(read_file(path))
    for line in data:
        seg = line.split()
        text = seg[-2::-1]
        # text = line
        # text = text.strip()
        words = text
        words = clean_words(words)
        for word in words:
            if word in vocabulary:
                vocabulary[word] = vocabulary[word] + 1
            else:
                vocabulary[word] = 1

    sorted_vocab = sorted(vocabulary.items(), key=lambda x: x[1])
    # sorted_vocab = [item for item in sorted_vocab if item[1] > freq_threshold]
    sorted_vocab = sorted_vocab[-1:-5001:-1]
    #
    # write vocablary

    texts = ['{} {}'.format(item[0], item[1]) for item in sorted_vocab]
    with open('vocab.new', mode='w', encoding='utf-8') as writer:
        writer.write('\n'.join(texts))


def create_corpus(paths, vocab_url):
    vocab, v_dict = load_vocab(vocab_url)
    data=[]
    for path in paths:
        data.extend(read_file(path))
    new_data = []
    for line in data:
        doc = []
        segs = line.strip().split()
        for seg in segs[1:]:
            word_id, freq = seg.split(':')
            tmp = [vocab[int(word_id)-1]] *int(freq)
            # print(tmp)
            doc.extend(tmp)
        doc = ' '.join(doc)
        new_data.append(doc)
    with open('Snippets', mode='w', encoding='utf-8') as writer:
        writer.write('\n'.join(new_data))


if __name__=='__main__':
    load_stopwords()
    paths = ['test.txt', 'train.txt']
    clean_vocab(paths)



    # load_data(['20news'])
    vocab, vocab_dict = load_vocab('vocab.new')
    pickle.dump(vocab_dict, open('vocab.pkl', 'wb'))
    load_files_nvdm('train.txt', vocab_dict)
    load_files_prodlda(paths[0], vocab_dict)

    paths = ['test.feat', 'train.feat']
    vocab_url=data_dir+'vocab.new'
    create_corpus(paths, 'vocab.new')
    produce_files(paths)

### Step2. EDA

In [ ]:
# Plotting
# import thr necessary libraries 
import seaborn as sns 
sns.set(rc={'figure.figsize':(11.7,8.27)})

# to ignore the warnings  
from warnings import filterwarnings 

# Plot for Word Count
temp1 = JP_py37[JP_py37["WC"] < 500]

sns.set_style('whitegrid') 
sns.distplot(temp1['WC'], kde = False, color ='red', bins = 100)

# Plot for Total Application
temp2 = JP_py37[JP_py37["total_app"] < 100]

sns.set_style('whitegrid') 
sns.distplot(temp2['total_app'], kde = False, color ='red', bins = 50)

# Plotting Correlation
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
sns.set(style='white', font_scale=1.2)

# Total Application
g = sns.JointGrid(data=JP_py37, x='WC', y='total_app', xlim=(0, 500), ylim=(50, 1000), height=10)
g = g.plot_joint(sns.regplot, color="xkcd:muted blue")
g = g.plot_marginals(sns.distplot, kde=False, bins=12, color="xkcd:bluey grey")
g.ax_joint.text(145, 95, 'r = 0.45, p < .001', fontstyle='italic')
plt.tight_layout()

# Female Application rate
g = sns.JointGrid(data=JP_py37, x='WC', y='women_rate', xlim=(0, 500), ylim=(0, 1), height=10)
g = g.plot_joint(sns.regplot, color="xkcd:muted blue")
g = g.plot_marginals(sns.distplot, kde=False, bins=12, color="xkcd:bluey grey")
g.ax_joint.text(145, 95, 'r = 0.45, p < .001', fontstyle='italic')
plt.tight_layout()

import matplotlib as mpl
import matplotlib.pyplot as plt
from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS
mpl.rcParams['figure.figsize']=(12.0,12.0)  
mpl.rcParams['font.size']=12            
mpl.rcParams['savefig.dpi']=100             
mpl.rcParams['figure.subplot.bottom']=.1 
stopwords = set(STOPWORDS)
wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stopwords,
                          max_words=1000,
                          max_font_size=40, 
                          random_state=100
                         ).generate(str(TM_py37.lemma_body))
print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud)
plt.axis('off')
plt.show();

### Step3. Topic Extraction (Embedded Topic Modeling, ETM) on PyTorch

In [ ]:
# starting from here when I get back

# Working Directory
import os
os.chdir(r'C:\Users\kwonr\Dropbox\JobAd_Gender\Analysis_TopicModeling\\Neural_Topic_Models')

# Check if GPU is set to work
import torch
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

#!/usr/bin/env python
# -*- encoding: utf-8 -*-

import sys
sys.argv=['']
del sys

import os
import re
import torch
import pickle
import argparse
import logging
import time
from models import ETM
import numpy as np
from tqdm import tqdm
from utils import *
from dataset import DocDataset
from multiprocessing import cpu_count
from multiprocessing import Pool
#from torch.utils.data import Dataset,DataLoader

parser = argparse.ArgumentParser('ETM topic model')
parser.add_argument('--taskname',type=str,default='cnews10k',help='Taskname e.g cnews10k')
parser.add_argument('--no_below',type=int,default=5,help='The lower bound of count for words to keep, e.g 10')
parser.add_argument('--no_above',type=float,default=0.005,help='The ratio of upper bound of count for words to keep, e.g 0.3')
parser.add_argument('--num_epochs',type=int,default=10,help='Number of iterations (set to 100 as default, but 1000+ is recommended.)')
parser.add_argument('--n_topic',type=int,default=20,help='Num of topics')
parser.add_argument('--bkpt_continue',type=bool,default=False,help='Whether to load a trained model as initialization and continue training.')
parser.add_argument('--use_tfidf',type=bool,default=False,help='Whether to use the tfidf feature for the BOW input')
parser.add_argument('--rebuild',type=bool,default=True,help='Whether to rebuild the corpus, such as tokenization, build dict etc.(default True)')
parser.add_argument('--batch_size',type=int,default=512,help='Batch size (default=512)')
parser.add_argument('--criterion',type=str,default='cross_entropy',help='The criterion to calculate the loss, e.g cross_entropy, bce_softmax, bce_sigmoid')
parser.add_argument('--emb_dim',type=int,default=300,help="The dimension of the latent topic vectors (default:300)")
parser.add_argument('--auto_adj',action='store_true',help='To adjust the no_above ratio automatically (default:rm top 20)')

args = parser.parse_args()

def main():
    global args
    taskname = args.taskname
    no_below = args.no_below
    no_above = args.no_above
    num_epochs = args.num_epochs
    n_topic = args.n_topic
    n_cpu = cpu_count()-2 if cpu_count()>2 else 2
    bkpt_continue = args.bkpt_continue
    use_tfidf = args.use_tfidf
    rebuild = args.rebuild
    batch_size = args.batch_size
    criterion = args.criterion
    n_topic = args.n_topic
    emb_dim = args.emb_dim
    auto_adj = args.auto_adj

    device = torch.device('cuda')
    docSet = DocDataset(taskname,no_below=no_below,no_above=no_above,rebuild=rebuild,use_tfidf=False)
    if auto_adj:
        no_above = docSet.topk_dfs(topk=20)
        docSet = DocDataset(taskname,no_below=no_below,no_above=no_above,rebuild=rebuild,use_tfidf=False)
    
    voc_size = docSet.vocabsize
    print('voc size:',voc_size)
    model = ETM(bow_dim=voc_size,n_topic=n_topic,taskname=taskname,device=device,emb_dim=emb_dim) #TBD_fc1
    model.train(train_data=docSet,batch_size=batch_size,test_data=docSet,num_epochs=num_epochs,log_every=10,beta=1.0,criterion=criterion)
    model.evaluate(test_data=docSet)
    save_name = f'./ckpt/ETM_{taskname}_tp{n_topic}_{time.strftime("%Y-%m-%d-%H-%M", time.localtime())}.ckpt'
    torch.save(model.vae.state_dict(),save_name)
    topic_vecs = model.vae.alpha.weight.detach().cpu().numpy()
    word_vecs = model.vae.rho.weight.detach().cpu().numpy()
    print('topic_vecs.shape:',topic_vecs.shape)
    print('word_vecs.shape:',word_vecs.shape)
    vocab = np.array([t[0] for t in sorted(list(docSet.dictionary.token2id.items()),key=lambda x: x[1])]).reshape(-1,1)
    topic_ids = np.array([f'TP{i}' for i in range(n_topic)]).reshape(-1,1)
    word_vecs = np.concatenate([vocab,word_vecs],axis=1)
    topic_vecs = np.concatenate([topic_ids,topic_vecs],axis=1)
    #save_name_tp = f'./ckpt/TpVec_ETM_{taskname}_tp{n_topic}_{time.strftime("%Y-%m-%d-%H-%M", time.localtime())}.emb'
    save_name_wd = f'./ckpt/WdVec_ETM_{taskname}_tp{n_topic}_{time.strftime("%Y-%m-%d-%H-%M", time.localtime())}.emb'
    n_instances = word_vecs.shape[0]+topic_vecs.shape[0]
    with open(save_name_wd,'w',encoding='utf-8') as wfp:
        wfp.write(f'{n_instances} {emb_dim}\n')
        wfp.write('\n'.join([' '.join(e) for e in word_vecs]+[' '.join(e) for e in topic_vecs]))
    from gensim.models import KeyedVectors
    w2v = KeyedVectors.load_word2vec_format(save_name_wd,binary=False)
    w2v.save(save_name.split('.')[0]+'.w2v')
    print(w2v.vocab.keys())
    #w2v.most_similar('你好')
    for i in range(n_topic):
        print(f'Most similar to Topic {i}')
        print(w2v.most_similar(f'TP{i}'))
    txt_lst, embeds = model.get_embed(train_data=docSet, num=1000)
    with open('topic_dist_etm.txt','w',encoding='utf-8') as wfp:
        for t,e in zip(txt_lst,embeds):
            wfp.write(f'{e}:{t}\n')
    pickle.dump({'txts':txt_lst,'embeds':embeds},open('etm_embeds.pkl','wb'))

if __name__ == "__main__":
    main()

### Step4. Feature Selection

In [ ]:
# Working Directory
import os
import pandas as pd
os.chdir(r'C:\Users\kwonr\Dropbox\JobAd_Gender\Analysis_Prediction')

# Read pickles
Job_Posting = pd.read_pickle("Job_Posting.pkl") 

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

# Skim
Job_Posting.describe()

# Normalize topic distribution values 
bodytopic = ['Body_Topic1', 'Body_Topic2', 'Body_Topic3', 'Body_Topic4', 'Body_Topic5', 'Body_Topic6', 'Body_Topic7', 'Body_Topic8', 'Body_Topic9', 'Body_Topic10', 
                      'Body_Topic11', 'Body_Topic12', 'Body_Topic13', 'Body_Topic14', 'Body_Topic15', 'Body_Topic16', 'Body_Topic17', 'Body_Topic18', 'Body_Topic19', 'Body_Topic20',
                      'Body_Topic21', 'Body_Topic22', 'Body_Topic23', 'Body_Topic24', 'Body_Topic25', 'Body_Topic26', 'Body_Topic27', 'Body_Topic28', 'Body_Topic29', 'Body_Topic30',
                      'Body_Topic31', 'Body_Topic32', 'Body_Topic33', 'Body_Topic34', 'Body_Topic35', 'Body_Topic36', 'Body_Topic37', 'Body_Topic38', 'Body_Topic39', 'Body_Topic40',
                      'Body_Topic41', 'Body_Topic42', 'Body_Topic43', 'Body_Topic44', 'Body_Topic45', 'Body_Topic46', 'Body_Topic47', 'Body_Topic48']


profiletopic = ['Profile_Topic1', 'Profile_Topic2', 'Profile_Topic3', 'Profile_Topic4', 'Profile_Topic5', 'Profile_Topic6', 'Profile_Topic7', 'Profile_Topic8', 'Profile_Topic9', 'Profile_Topic10', 
                      'Profile_Topic11', 'Profile_Topic12', 'Profile_Topic13', 'Profile_Topic14', 'Profile_Topic15', 'Profile_Topic16', 'Profile_Topic17', 'Profile_Topic18', 'Profile_Topic19', 'Profile_Topic20',
                      'Profile_Topic21', 'Profile_Topic22', 'Profile_Topic23', 'Profile_Topic24', 'Profile_Topic25', 'Profile_Topic26', 'Profile_Topic27', 'Profile_Topic28', 'Profile_Topic29', 'Profile_Topic30',
                      'Profile_Topic31', 'Profile_Topic32', 'Profile_Topic33', 'Profile_Topic34', 'Profile_Topic35', 'Profile_Topic36', 'Profile_Topic37', 'Profile_Topic38', 'Profile_Topic39', 'Profile_Topic40',
                      'Profile_Topic41', 'Profile_Topic42', 'Profile_Topic43', 'Profile_Topic44', 'Profile_Topic45', 'Profile_Topic46']

Job_Posting[bodytopic] = Job_Posting[bodytopic].div(Job_Posting.sum_prob_body, axis = 0)
Job_Posting['sum_prob_body'] = Job_Posting.loc[:,'Body_Topic1':'Body_Topic48'].sum(axis = 1)

Job_Posting[profiletopic] = Job_Posting[profiletopic].div(Job_Posting.sum_prob_profile, axis = 0)
Job_Posting['sum_prob_profile'] = Job_Posting.loc[:,'Profile_Topic1':'Profile_Topic46'].sum(axis = 1)

#Drop redundant columns
Job_Posting.drop(['sum_prob_body', 'sum_prob_profile'], axis=1, inplace=True)

# Subsetting (only extract columns needed)
import numpy as np
loc = Job_Posting.columns.get_loc
x = Job_Posting.iloc[:, np.r_[loc('job_role'):loc('Profile_Topic46')+1, loc('women_rate')]]

# Check missing values & Drop columns having NA/NaN values
x.isnull().sum()
x.dropna(inplace = True)

y = x['women_rate'].values

# Generate a classification threshold
print("lower 10% is : ", np.quantile(y, .1))
print("lower 20% is : ", np.quantile(y, .2))
print("lower 30% is : ", np.quantile(y, .3))
print("lower 40% is : ", np.quantile(y, .4))
print("lower 50% is : ", np.quantile(y, .5))
print("lower 60% is : ", np.quantile(y, .6))
print("lower 70% is : ", np.quantile(y, .7))
print("lower 80% is : ", np.quantile(y, .8))
print("lower 90% is : ", np.quantile(y, .9))

y[y >= 0.369942197] = 1
y[y < 0.369942197] = 0

from collections import Counter
B = Counter(y)
B
# Cleaning 

# number_of_vacancy
x['D_vacancy'] = np.where(x['number_of_vacancy'] >= 10, 1, 0)

#job_role & api_job_gender

# One-hot encoding 
x_dummy1 = pd.get_dummies(x.job_role, prefix='job_role')
x_dummy2 = pd.get_dummies(x.api_job_gender, prefix='api_job_gender')

print(x_dummy1.head())
print(x_dummy2.head())

# Join the encoded df (later, I should avoid dummy trap)
x = x.join(x_dummy1)
x = x.join(x_dummy2)

# hiring_process

x['hiring_process_1'] = np.where(x['hiring_process'].str.contains('1'), 1, 0)
x['hiring_process_2'] = np.where(x['hiring_process'].str.contains('2'), 1, 0)
x['hiring_process_3'] = np.where(x['hiring_process'].str.contains('3'), 1, 0)
x['hiring_process_4'] = np.where(x['hiring_process'].str.contains('4'), 1, 0)
x['hiring_process_5'] = np.where(x['hiring_process'].str.contains('5'), 1, 0)

# Drop columns as it is now encoded & meaningless columns
x = x.drop(['hiring_process', 'number_of_vacancy', 'job_role', 'api_job_gender',
            'job_role_12', 'api_job_gender_-1', 'hiring_process_1', 'women_rate'],axis = 1)

#Data rype
x.info()

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.75, test_size=0.25, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# Check the balance of the splits on y_
print(np.mean(y_train))
print(np.mean(y_test))

# Feature Scaling 
from sklearn.preprocessing import MinMaxScaler

# define min max scaler
scaler = MinMaxScaler()

# transform data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# Cleaning 

# number_of_vacancy
x['D_vacancy'] = np.where(x['number_of_vacancy'] >= 10, 1, 0)

#job_role & api_job_gender

# One-hot encoding 
x_dummy1 = pd.get_dummies(x.job_role, prefix='job_role')
x_dummy2 = pd.get_dummies(x.api_job_gender, prefix='api_job_gender')

print(x_dummy1.head())
print(x_dummy2.head())

# Join the encoded df (later, I should avoid dummy trap)
x = x.join(x_dummy1)
x = x.join(x_dummy2)

# hiring_process

x['hiring_process_1'] = np.where(x['hiring_process'].str.contains('1'), 1, 0)
x['hiring_process_2'] = np.where(x['hiring_process'].str.contains('2'), 1, 0)
x['hiring_process_3'] = np.where(x['hiring_process'].str.contains('3'), 1, 0)
x['hiring_process_4'] = np.where(x['hiring_process'].str.contains('4'), 1, 0)
x['hiring_process_5'] = np.where(x['hiring_process'].str.contains('5'), 1, 0)

# Drop columns as it is now encoded & meaningless columns
x = x.drop(['hiring_process', 'number_of_vacancy', 'job_role', 'api_job_gender',
            'job_role_12', 'api_job_gender_-1', 'hiring_process_1', 'women_rate'],axis = 1)

#Data rype
x.info()

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.75, test_size=0.25, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# Check the balance of the splits on y_
print(np.mean(y_train))
print(np.mean(y_test))

# Feature Scaling 
from sklearn.preprocessing import MinMaxScaler

# define min max scaler
scaler = MinMaxScaler()

# transform data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
import matplotlib.pyplot as plt

feature_names = x.columns

# Forward
from sklearn.linear_model import LinearRegression
mlxtend_linear_forward = SFS(LinearRegression(), 
                             k_features=30, 
                             forward=True, 
                             floating=False, 
                             scoring='neg_mean_squared_error',
                             cv=5)

mlxtend_linear_forward.fit(x, y, custom_feature_names=feature_names)

fig1 = plot_sfs(mlxtend_linear_forward.get_metric_dict(), kind='std_err')

plt.title('Sequential Forward Selection (w. std_err)')
plt.grid()
plt.show()

# Backward
mlxtend_linear_backward = SFS(LinearRegression(), 
                             k_features=30, 
                             forward=False, 
                             floating=False, 
                             scoring='neg_mean_squared_error',
                             cv=5)

mlxtend_linear_backward.fit(x, y, custom_feature_names=feature_names)

fig2 = plot_sfs(mlxtend_linear_backward.get_metric_dict(), kind='std_err')

plt.title('Sequential backward Selection (w. std_err)')
plt.grid()
plt.show()

# Feature Selection - Wrapper Methods
from sklearn.feature_selection import SequentialFeatureSelector as sfs

# linear regression - forward
feature_names = x.columns

#for digit in range(5,31):
sfs_linear_forward = sfs(LinearRegression(),
                         n_features_to_select=digit,
                         direction='forward',
                         cv=5,
                         n_jobs=-1)

sfs_linear_forward.fit(x, y)

# Print Results - Forward Selection
print("Features selected by forward sequential selection: " 
      f"{feature_names[sfs_linear_forward.get_support()]}")

print(sfs_linear_forward.transform(x).shape)

# linear regression - backward
#for digit in range(5,31):
feature_names = x.columns

from sklearn.linear_model import LinearRegression
sfs_linear_backward = sfs(LinearRegression(),
                         n_features_to_select=30,
                         direction='backward',
                         cv=5,
                         n_jobs=-1)

sfs_linear_backward.fit(x, y)

# Print Results - Forward Selection
print("Features selected by backward sequential selection: " 
      f"{feature_names[sfs_linear_backward.get_support()]}")

print(sfs_linear_backward.transform(x).shape)

### Step5. Prediction

In [ ]:
# Ridge
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

steps = [
    ('scalar', StandardScaler()),
    ('model', Ridge(alpha=3.8, fit_intercept=True))
]

ridge_pipe = Pipeline(steps)
ridge_pipe.fit(X_train, y_train)

# Predicting Cross Validation Score the Test set results
cv_ridge = cross_val_score(estimator = ridge_pipe, X = X_train, y = y_train, cv = 10)

# Predicting R2 Score the Test set results
y_pred_ridge_test = ridge_pipe.predict(X_test)

# Predicting RMSE the Test set results
rmse_ridge = (np.sqrt(mean_squared_error(y_test, y_pred_ridge_test)))
print('CV: ', cv_ridge.mean())
print("RMSE: ", rmse_ridge)

# Lasso
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

steps = [
    ('scalar', StandardScaler()),
    ('model', Lasso(alpha=0.012, fit_intercept=True, max_iter=3000))
]

lasso_pipe = Pipeline(steps)
lasso_pipe.fit(X_train, y_train)

# Predicting Cross Validation Score
cv_lasso = cross_val_score(estimator = lasso_pipe, X = X_train, y = y_train, cv = 10)

# Predicting R2 Score the Test set results
y_pred_lasso_test = lasso_pipe.predict(X_test)

# Predicting RMSE the Test set results
rmse_lasso = (np.sqrt(mean_squared_error(y_test, y_pred_lasso_test)))
print('CV: ', cv_lasso.mean())
print("RMSE: ", rmse_lasso)

# Decision Tree
#Fitting the Decision Tree Regression Model to the dataset
from sklearn.tree import DecisionTreeRegressor
regressor_dt = DecisionTreeRegressor(random_state = 0)
regressor_dt.fit(X_train, y_train)
# Predicting Cross Validation Score
cv_dt = cross_val_score(estimator = regressor_dt, X = X_train, y = y_train, cv = 10)

# Predicting R2 Score the Test set results
y_pred_dt_test = regressor_dt.predict(X_test)

# Predicting RMSE the Test set results
rmse_dt = (np.sqrt(mean_squared_error(y_test, y_pred_dt_test)))
print('CV: ', cv_dt.mean())
print("RMSE: ", rmse_dt)

# Fitting the Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
regressor_rf = RandomForestRegressor(n_estimators = 500, random_state = 0)
regressor_rf.fit(X_train, y_train)

# Predicting Cross Validation Score
cv_rf = cross_val_score(estimator = regressor_rf, X = X_train, y = y_train, cv = 10)

# Predicting R2 Score the Test set results
y_pred_rf_test = regressor_rf.predict(X_test)

# Predicting RMSE the Test set results
rmse_rf = (np.sqrt(mean_squared_error(y_test, y_pred_rf_test)))
print('CV: ', cv_rf.mean())
print("RMSE: ", rmse_rf)

models = [('Linear Regression', rmse_linear, cv_linear.mean()),
          ('Ridge Regression', rmse_ridge, cv_ridge.mean()),
          ('Lasso Regression', rmse_lasso, cv_lasso.mean()),
          ('Decision Tree Regression', rmse_dt, cv_dt.mean()),
         ]

predict = pd.DataFrame(data = models, columns=['Model', 'RMSE', 'Cross-Validation'])
predict

# Instantiate & Fitting
linear_base = LogisticRegression(solver='liblinear')
linear_topic = LogisticRegression(solver='liblinear')
linear_LIWC = LogisticRegression(solver='liblinear')
linear_TLIWC = LogisticRegression(solver='liblinear')
linear_LIWCB = LogisticRegression(solver='liblinear')
linear_LIWCTopicB = LogisticRegression(solver='liblinear')

# Fit
linear_base.fit(train_base, y_train)
linear_topic.fit(train_topic, y_train)
linear_LIWC.fit(train_LIWC, y_train)
linear_TLIWC.fit(train_TLIWC, y_train)
linear_LIWCB.fit(train_LIWCB, y_train)
linear_LIWCTopicB.fit(train_LIWCTopicB, y_train)

# Predictions on the test dataset
y_pred_linear_base = pd.DataFrame(linear_base.predict(test_base))
y_pred_linear_topic = pd.DataFrame(linear_topic.predict(test_topic))
y_pred_linear_LIWC = pd.DataFrame(linear_LIWC.predict(test_LIWC))
y_pred_linear_TLIWC = pd.DataFrame(linear_TLIWC.predict(test_TLIWC))
y_pred_linear_LIWCB = pd.DataFrame(linear_LIWCB.predict(test_LIWCB))
y_pred_linear_LIWCTopicB = pd.DataFrame(linear_LIWCTopicB.predict(test_LIWCTopicB))

# Performance

print(accuracy_score(y_test, y_pred_linear_base))
print(accuracy_score(y_test, y_pred_linear_topic))
print(accuracy_score(y_test, y_pred_linear_LIWC))
print(accuracy_score(y_test, y_pred_linear_TLIWC))
print(accuracy_score(y_test, y_pred_linear_LIWCB))
print(accuracy_score(y_test, y_pred_linear_LIWCTopicB))

print(metrics.classification_report(y_test, y_pred_linear_base))
print(metrics.classification_report(y_test, y_pred_linear_topic))
print(metrics.classification_report(y_test, y_pred_linear_LIWC))
print(metrics.classification_report(y_test, y_pred_linear_TLIWC))
print(metrics.classification_report(y_test, y_pred_linear_LIWCB))
print(metrics.classification_report(y_test, y_pred_linear_LIWCTopicB))

# SVM
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

def evaluate(model, X_test, y_test):
    #Predict
    predictions = model.predict(X_test)
    #probs = pd.DataFrame(model.predict_proba(X_test))
    # Store metrics
    rf_accuracy = metrics.accuracy_score(y_test, predictions)
    #rf_roc_auc = metrics.roc_auc_score(y_test, probs[1])
    rf_confus_matrix = metrics.confusion_matrix(y_test, predictions)
    rf_classification_report = metrics.classification_report(y_test, predictions)
    rf_precision = metrics.precision_score(y_test, predictions, pos_label=1)
    rf_recall = metrics.recall_score(y_test, predictions, pos_label=1)
    rf_f1 = metrics.f1_score(y_test, predictions, pos_label=1)
    
    print('Model Performance')
    print('Accuracy = {:0.4f}%.'.format(rf_accuracy))
    #print('rf_roc_auc = {:0.4f}%.'.format(rf_roc_auc))
    print(rf_confus_matrix)
    print(rf_classification_report)
    print('rf_precision = {:0.4f}%.'.format(rf_precision))
    print('rf_recall = {:0.4f}%.'.format(rf_recall))
    print('rf_f1 = {:0.4f}%.'.format(rf_f1))
    return rf_accuracy


# FNN
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
random.seed(1234)

#Sigmoid is used as the activation function
def sigmoid(x):
    return 1/(1 + np.exp(-x))

#Derivative of the sigmoid function
def sigmoid_prime(x):
    return sigmoid(x)*(1.0 - sigmoid(x))

class NeuralNetwork(object):
    
    def __init__(self, architecture):
        #architecture - numpy array with ith element representing the number of neurons in the ith layer.
        
        #Initialize the network architecture
        self.L = architecture.size - 1 #The index of the last layer L
        self.n = architecture #n stores the number of neurons in each layer
        self.input_size = self.n[0] #input_size is the number of neurons in the first layer
        self.output_size = self.n[self.L] #output_size is the number of neurons in the last layer
        
        #Parameters will store the network parameters, i.e. the weights and biases
        self.parameters = {}
        
        #Initialize the network weights and biases:
        for i in range (1, self.L + 1): 
            #Initialize weights to small random values
            self.parameters['W' + str(i)] = np.random.randn(self.n[i], self.n[i - 1]) * 0.01
            
            #Initialize rest of the parameters to 1
            self.parameters['b' + str(i)] = np.ones((self.n[i], 1))
            self.parameters['z' + str(i)] = np.ones((self.n[i], 1))
            self.parameters['a' + str(i)] = np.ones((self.n[i], 1))
        
        #As we started the loop from 1, we haven't initialized a[0]:
        self.parameters['a0'] = np.ones((self.n[i], 1))
        
        #Initialize the cost:
        self.parameters['C'] = 1
        
        #Create a dictionary for storing the derivatives:
        self.derivatives = {}
        
        #Learning rate
        self.alpha = 0.01
            
    def forward_propagate(self, X):
        #Note that X here, is just one training example
        self.parameters['a0'] = X
        
        #Calculate the activations for every layer l
        for l in range(1, self.L + 1):
            self.parameters['z' + str(l)] = np.add(np.dot(self.parameters['W' + str(l)], self.parameters['a' + str(l - 1)]), self.parameters['b' + str(l)])
            self.parameters['a' + str(l)] = sigmoid(self.parameters['z' + str(l)])
        
    def compute_cost(self, y):
        self.parameters['C'] = -(y*np.log(self.parameters['a' + str(self.L)]) + (1-y)*np.log( 1 - self.parameters['a' + str(self.L)]))
    
    def compute_derivatives(self, y):
        #Partial derivatives of the cost function with respect to z[L], W[L] and b[L]:        
        #dzL
        self.derivatives['dz' + str(self.L)] = self.parameters['a' + str(self.L)] - y
        #dWL
        self.derivatives['dW' + str(self.L)] = np.dot(self.derivatives['dz' + str(self.L)], np.transpose(self.parameters['a' + str(self.L - 1)]))
        #dbL
        self.derivatives['db' + str(self.L)] = self.derivatives['dz' + str(self.L)]

        #Partial derivatives of the cost function with respect to z[l], W[l] and b[l]
        for l in range(self.L-1, 0, -1):
            self.derivatives['dz' + str(l)] = np.dot(np.transpose(self.parameters['W' + str(l + 1)]), self.derivatives['dz' + str(l + 1)])*sigmoid_prime(self.parameters['z' + str(l)])
            self.derivatives['dW' + str(l)] = np.dot(self.derivatives['dz' + str(l)], np.transpose(self.parameters['a' + str(l - 1)]))
            self.derivatives['db' + str(l)] = self.derivatives['dz' + str(l)]
            
    def update_parameters(self):
        for l in range(1, self.L+1):
            self.parameters['W' + str(l)] -= self.alpha*self.derivatives['dW' + str(l)]
            self.parameters['b' + str(l)] -= self.alpha*self.derivatives['db' + str(l)]
        
    def predict(self, x):
        self.forward_propagate(x)
    
        return self.parameters['a' + str(self.L)]
        
    def fit(self, X, Y, num_iter):
        for iter in range(0, num_iter):
            c = 0
            acc = 0
            n_c = 0
            for i in range(0, X.shape[0]):
              x = X[i].reshape((X[i].size, 1))
              y = Y[i]
              self.forward_propagate(x)
              self.compute_cost(y)
              c += self.parameters['C'] 
              y_pred = self.predict(x)
              y_pred = (y_pred > 0.5)
              if y_pred == y:
                  n_c += 1
              self.compute_derivatives(y)
              self.update_parameters()
            
            c = c/X.shape[0]
            acc = (n_c/X.shape[0])*100
            print('Iteration: ', iter)
            print("Cost: ", c)
            print("Accuracy:", acc)
            
#Defining the model architecture
architecture = np.array([80, 100, 1])

#Creating the classifier
classifier = NeuralNetwork(architecture)

#Training the classifier
classifier.fit(X_train, y_train, 100)

#Predicting the test set results:
acc = 0
n_c = 0
for i in range(0, X_test.shape[0]):
  x = X_test[i].reshape((X_test[i].size, 1))
  y = y_test[i]
  y_pred = classifier.predict(x)
  y_pred = (y_pred > 0.5)
  #print('Expected: %d Got: %d' %(y, y_pred))
  if y_pred == y:
      n_c += 1

acc = (n_c/X_test.shape[0])*100
print("Test Accuracy", acc)

# First create the base model to tune
SVM_classifier = SVC(kernel = 'linear', gamma = 'auto', C=2,probability = True).fit(X_train, y_train)
base_accuracy = evaluate(SVM_classifier, X_test, y_test)

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
SVM_random = RandomizedSearchCV(estimator = SVM_classifier, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=2, random_state=42, n_jobs = -1)
SVM_random.fit(X_train, y_train)

best_SVM = SVM_random.best_estimator_
SVM_accuracy = evaluate(best_SVM, X_test, y_test)

# Random Forest
from sklearn.model_selection import train_test_split
from random import sample
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from pprint import pprint
# https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
rf = RandomForestClassifier(random_state = 42)

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 200, num = 1)]

# Number of features to consider at every split
max_features = ['auto']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2,5,10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'max_features': max_features,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

def evaluate(model, X_test, y_test):
    #Predict
    predictions = model.predict(X_test)
    probs = pd.DataFrame(model.predict_proba(X_test))
    # Store metrics
    rf_accuracy = metrics.accuracy_score(y_test, predictions)
    rf_roc_auc = metrics.roc_auc_score(y_test, probs[1])
    rf_confus_matrix = metrics.confusion_matrix(y_test, predictions)
    rf_classification_report = metrics.classification_report(y_test, predictions)
    rf_precision = metrics.precision_score(y_test, predictions, pos_label=1)
    rf_recall = metrics.recall_score(y_test, predictions, pos_label=1)
    rf_f1 = metrics.f1_score(y_test, predictions, pos_label=1)
    
    print('Model Performance')
    print('Accuracy = {:0.4f}%.'.format(rf_accuracy))
    print('rf_roc_auc = {:0.4f}%.'.format(rf_roc_auc))
    print(rf_confus_matrix)
    print(rf_classification_report)
    print('rf_precision = {:0.4f}%.'.format(rf_precision))
    print('rf_recall = {:0.4f}%.'.format(rf_recall))
    print('rf_f1 = {:0.4f}%.'.format(rf_f1))
    
    return rf_accuracy

# Use the random grid to search for best hyperparameters
# First create the base model to tune

print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

base_model = RandomForestClassifier(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = base_model, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)

# Grid Search
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
print(rf_random.best_params_)

max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

param_grid = {
    'bootstrap': [True],
    'max_depth': max_depth,
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [int(x) for x in np.linspace(start = 20, stop = 200, num = 1)]
}
# Create a based model
rf = RandomForestClassifier(n_estimators = 10, random_state = 42)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 10, n_jobs = -1, verbose = 2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_test, y_test)

# Get numerical feature importances
importances_Topic = list(grid_search.best_estimator_.feature_importances_)

# List of tuples with variable and importance
X_TP16_Topic = x
Topic_importances = [(feature, round(importances_Topic, 4)) for feature, importances_Topic in zip(X_TP16_Topic, importances_Topic)]

# Sort the feature importances by most important first
Topic_importances = sorted(Topic_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:80} Importance: {}'.format(*pair)) for pair in Topic_importances];

# Cross-Validation for LASSO

# Generate a new dataframe (subsetting)
LASSO = x[['Analytic', 'b_topic11', 'work', 'affect', 'achieve']]

# Renaming for brevity
LASSO.rename(columns = {'Analytic':'X1',
                       'b_topic11':'X2',
                       'work':'X3',
                       'affect':'X4',
                       'achieve':'X5'}, inplace = True)

# Make a superset (squared term = 8)
LASSO = LASSO.assign(X1X1 = LASSO.X1 ** 2)
LASSO = LASSO.assign(X2X2 = LASSO.X2 ** 2)
LASSO = LASSO.assign(X3X3 = LASSO.X3 ** 2)
LASSO = LASSO.assign(X4X4 = LASSO.X4 ** 2)
LASSO = LASSO.assign(X5X5 = LASSO.X5 ** 2)
#LASSO = LASSO.assign(X6X6 = LASSO.X6 ** 2)
#LASSO = LASSO.assign(X7X7 = LASSO.X7 ** 2)
#LASSO = LASSO.assign(X8X8 = LASSO.X8 ** 2)

# Make a superset (interaction term = 8C2)
LASSO = LASSO.assign(X1X2 = LASSO.X1 * LASSO.X2)
LASSO = LASSO.assign(X1X3 = LASSO.X1 * LASSO.X3)
LASSO = LASSO.assign(X1X4 = LASSO.X1 * LASSO.X4)
LASSO = LASSO.assign(X1X5 = LASSO.X1 * LASSO.X5)
#LASSO = LASSO.assign(X1X6 = LASSO.X1 * LASSO.X6)
#LASSO = LASSO.assign(X1X7 = LASSO.X1 * LASSO.X7)
#LASSO = LASSO.assign(X1X8 = LASSO.X1 * LASSO.X8)
LASSO = LASSO.assign(X2X3 = LASSO.X2 * LASSO.X3)
LASSO = LASSO.assign(X2X4 = LASSO.X2 * LASSO.X4)
LASSO = LASSO.assign(X2X5 = LASSO.X2 * LASSO.X5)
#LASSO = LASSO.assign(X2X6 = LASSO.X2 * LASSO.X6)
#LASSO = LASSO.assign(X2X7 = LASSO.X2 * LASSO.X7)
#LASSO = LASSO.assign(X2X8 = LASSO.X2 * LASSO.X8)
LASSO = LASSO.assign(X3X4 = LASSO.X3 * LASSO.X4)
LASSO = LASSO.assign(X3X5 = LASSO.X3 * LASSO.X5)
#LASSO = LASSO.assign(X3X6 = LASSO.X3 * LASSO.X6)
#LASSO = LASSO.assign(X3X7 = LASSO.X3 * LASSO.X7)
#LASSO = LASSO.assign(X3X8 = LASSO.X3 * LASSO.X8)
LASSO = LASSO.assign(X4X5 = LASSO.X4 * LASSO.X5)
#LASSO = LASSO.assign(X4X6 = LASSO.X4 * LASSO.X6)
#LASSO = LASSO.assign(X4X7 = LASSO.X4 * LASSO.X7)
#LASSO = LASSO.assign(X4X8 = LASSO.X4 * LASSO.X8)
#LASSO = LASSO.assign(X5X6 = LASSO.X5 * LASSO.X6)
#LASSO = LASSO.assign(X5X7 = LASSO.X5 * LASSO.X7)
#LASSO = LASSO.assign(X5X8 = LASSO.X5 * LASSO.X8)
#LASSO = LASSO.assign(X6X7 = LASSO.X6 * LASSO.X7)
#LASSO = LASSO.assign(X6X8 = LASSO.X6 * LASSO.X8)
#LASSO = LASSO.assign(X7X8 = LASSO.X7 * LASSO.X8)

# Cross-validation for LASSO Hyperparameter Selection
from sklearn.linear_model import LassoCV
from sklearn.model_selection import RepeatedKFold

# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=1)
# define model
alphas = np.logspace(-12, 1, 400)
LASSO_model = LassoCV(alphas=alphas, cv=cv, n_jobs=-1)
# fit model
LASSO_model.fit(X_train, y_train)
# summarize chosen configuration
print('alpha: %f' % LASSO_model.alpha_) # alpha: 0.003028

# Visualization
from yellowbrick.datasets import load_concrete
from yellowbrick.regressor import AlphaSelection

# Instantiate the linear model and visualizer
visualizer = AlphaSelection(LASSO_model)
visualizer.fit(LASSO, y)
visualizer.show()

# Feature Selection

# Sub -sample
# Sample Splitting 
LASSO1 = LASSO.iloc[0:1000, :]
LASSO2 = LASSO.iloc[1000:2000, :]
LASSO3 = LASSO.iloc[2000:3000, :]
LASSO4 = LASSO.iloc[3000:4000, :]
LASSO5 = LASSO.iloc[4000:5000, :]
LASSO6 = LASSO.iloc[5000:6000, :]
LASSO7 = LASSO.iloc[6000:7000, :]
LASSO8 = LASSO.iloc[7000:8000, :]
LASSO9 = LASSO.iloc[8000:9000, :]
LASSO10 = LASSO.iloc[9000:10358, :]

print(len(LASSO1))
print(len(LASSO2))
print(len(LASSO3))
print(len(LASSO4))
print(len(LASSO5))
print(len(LASSO6))
print(len(LASSO7))
print(len(LASSO8))
print(len(LASSO9))
print(len(LASSO10))

y1 = y[0:1000]
y2 = y[1000:2000]
y3 = y[2000:3000]
y4 = y[3000:4000]
y5 = y[4000:5000]
y6 = y[5000:6000]
y7 = y[6000:7000]
y8 = y[7000:8000]
y9 = y[8000:9000]
y10 = y[9000:10358]

print(len(y1))
print(len(y2))
print(len(y3))
print(len(y4))
print(len(y5))
print(len(y6))
print(len(y7))
print(len(y8))
print(len(y9))
print(len(y10))

# Full sample
# define model
alphas = np.logspace(-12, 1, 400)
LASSO_model1 = LassoCV(alphas=alphas, cv=cv, n_jobs=-1)
LASSO_model2 = LassoCV(alphas=alphas, cv=cv, n_jobs=-1)
LASSO_model3 = LassoCV(alphas=alphas, cv=cv, n_jobs=-1)
LASSO_model4 = LassoCV(alphas=alphas, cv=cv, n_jobs=-1)
LASSO_model5 = LassoCV(alphas=alphas, cv=cv, n_jobs=-1)
LASSO_model6 = LassoCV(alphas=alphas, cv=cv, n_jobs=-1)
LASSO_model7 = LassoCV(alphas=alphas, cv=cv, n_jobs=-1)
LASSO_model8 = LassoCV(alphas=alphas, cv=cv, n_jobs=-1)
LASSO_model9 = LassoCV(alphas=alphas, cv=cv, n_jobs=-1)
LASSO_model10 = LassoCV(alphas=alphas, cv=cv, n_jobs=-1)

# fit model
LASSO_model1.fit(LASSO1, y1)
LASSO_model2.fit(LASSO2, y2)
LASSO_model3.fit(LASSO3, y3)
LASSO_model4.fit(LASSO4, y4)
LASSO_model5.fit(LASSO5, y5)
LASSO_model6.fit(LASSO6, y6)
LASSO_model7.fit(LASSO7, y7)
LASSO_model8.fit(LASSO8, y8)
LASSO_model9.fit(LASSO9, y9)
LASSO_model10.fit(LASSO10, y10)

#Save coefficients
lasso_coeff = pd.DataFrame()
lasso_coeff['Full sample'] = pd.Series(LASSO_model.coef_)
lasso_coeff['subsample1'] = pd.Series(LASSO_model1.coef_)
lasso_coeff['subsample2'] = pd.Series(LASSO_model2.coef_)
lasso_coeff['subsample3'] = pd.Series(LASSO_model3.coef_)
lasso_coeff['subsample4'] = pd.Series(LASSO_model4.coef_)
lasso_coeff['subsample5'] = pd.Series(LASSO_model5.coef_)
lasso_coeff['subsample6'] = pd.Series(LASSO_model6.coef_)
lasso_coeff['subsample7'] = pd.Series(LASSO_model7.coef_)
lasso_coeff['subsample8'] = pd.Series(LASSO_model8.coef_)
lasso_coeff['subsample9'] = pd.Series(LASSO_model9.coef_)
lasso_coeff['subsample10'] = pd.Series(LASSO_model10.coef_)
lasso_coeff

import matplotlib.pyplot as plt
import seaborn as sns

ax = sns.heatmap(lasso_coeff, 
                 linewidths=.5, 
                 robust=True ,
                 annot_kws = {'size':14}, 
                 yticklabels=LASSO.columns, 
                 center=0, 
                 cmap='coolwarm')
ax.tick_params(labelsize=14)
ax.figure.set_size_inches((12, 10))

#Bootstrap samples
from sklearn.utils import resample

#organizing dataset 
#add 'y' array as new column in DataFrame
values = LASSO
values['y'] = y.tolist()

# Bootstrap configuration
n_iterations = 30  #Nso. of bootstrap samples to be repeated (created)
n_size = int(len(values) * 0.80) #Size of sample, picking only 80% of the given data in every bootstrap sample

#Lets run Bootstrap
lasso_boostrap0 = pd.DataFrame()
for i in range(n_iterations):

    #prepare train & test sets
    train = resample(values, n_samples = n_size) #Sampling with replacement..whichever is not used in training data will be used in test data
    test = np.array([x for x in values if x not in train]) #picking rest of the data not considered in training sample
    
    # define model evaluation method
    cv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=1)
    # define model
    alphas = np.logspace(-12, 1, 400)
    LASSO_model0 = LassoCV(alphas=alphas, cv=cv, n_jobs=-1)
    # fit model
    LASSO_model0.fit(train.iloc[:,:-1], train.iloc[:,-1])
    #coefficient
    result0 = pd.DataFrame(LASSO_model0.coef_)
    lasso_boostrap0 = pd.concat([lasso_boostrap0, result0], axis=1)
    
lasso_boostrap0['mean_estimate'] = lasso_boostrap0.mean(axis=1)